# Prerequisites

In [1]:
import sys

sys.path.append("..")

In [2]:
%load_ext autoreload
%autoreload 2

import argparse
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from IPython.display import display
from torchvision.models import resnet18, resnet50
from tqdm.notebook import tqdm

from dataset import (
    HWSet,
    normalize_hw,
    normalize_hw_mask,
    normalize_inv_hw,
    normalize_inv_hw_mask,
    transform_shared,
    get_dloader
)
from util import eval_step, get_obj_score, get_performance, get_saliency

sns.set_theme()

C:\Users\runes\.conda\envs\context\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [3]:
data_dir = "../data"
class_legend = ("Siberian Husky", "Grey Wolf")
model_types = {"r18": "ResNet 18", "r50": "Resnet 50"}

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")


def get_model(model_type, device="cpu", seed=191510):
    torch.manual_seed(seed)
    if model_type == "r18":
        model = resnet18(weights=None)
    elif model_type == "r50":
        model = resnet50(weights=None)
    model.fc = torch.nn.Linear(
        in_features=model.fc.in_features, out_features=len(class_legend), bias=True
    )
    model.to(device)
    return model

Using device: cuda


# Timing and testing

In [4]:
dloader = get_dloader('train', 16, data_dir=data_dir, noise=True, num_workers=0)

In [5]:
%%timeit
for imgs, labels, masks, noise in dloader:
    pass

51.3 s ± 1.43 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
dloader = get_dloader('train', 16, data_dir=data_dir, noise=True, num_workers=2)

In [7]:
%%timeit
for imgs, labels, masks, noise in dloader:
    pass

54.6 s ± 5.97 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
dloader = get_dloader('train', 16, data_dir=data_dir, noise=True, num_workers=4)

In [9]:
%%timeit
for imgs, labels, masks, noise in dloader:
    pass

OSError: [Errno 22] Invalid argument